# Замеры скорости для PyPy

Так как для запуска PyPy отдельно от Питона нужен отдельный интерпретатор, блокнот тоже взят отдельный.

In [1]:
from annotation import jit
from timeit import timeit, repeat
from json import dumps

import numpy as np


In [3]:
results = {}

## Усреднённые значения вызовов на простой функции

In [4]:
def pypy_sum(x: int, y: int) -> int:
    res: int = x + y
    return res

In [5]:
pypy_exec_time = repeat(lambda: pypy_sum(2, 2), repeat=10, number=1000000)
print(pypy_exec_time)

[0.09988090000115335, 0.09813860000576824, 0.09835220000240952, 0.11718719999771565, 0.09999650000827387, 0.09985900000901893, 0.0995898999972269, 0.09989769995445386, 0.09874079999281093, 0.09859180002240464]


In [6]:
xs = range(1, 11)
results["sum"] = [list(xs), pypy_exec_time]


## Функция вычисления экспоненты с помощью разложения в ряд Тейлора

In [8]:
def pypy_exp(x: float) -> float:
    res: float = 0
    threshold: float = 1e-30
    delta: float = 1
    elements: int = 0
    while delta > threshold:
        elements = elements + 1
        delta = delta * x / elements
    while elements >= 0:
        res += delta
        delta = delta * elements / x
        elements -= 1
    return res


In [11]:
arg = 250
print(f"exp({arg}) = {pypy_exp(arg):.30f}")
args = range(10, 260, 10)
pypy_exp_time = []
for arg in args:
    pypy_exp_time.append(timeit(lambda: pypy_exp(arg), number=10000))
print(pypy_exp_time)

exp(250) = 3746454614502660877998657881484689260451454624001099543290316630153610787704025897267034669677141296546840576.000000000000000000000000000000
[0.12120669998694211, 0.1676558000035584, 0.23789669998222962, 0.2760334999766201, 0.32595530000980943, 0.35333519999403507, 0.4003392999875359, 0.4515677000163123, 0.5425614000414498, 0.555046699999366, 0.6281444999622181, 0.6689620999968611, 0.690308700024616, 0.7371551000396721, 0.7855007000034675, 0.8472960999934003, 0.9347611999837682, 0.9237144000362605, 0.982869400002528, 1.0346267000422813, 1.077914499968756, 1.1484616000088863, 1.1625329999951646, 1.250912800023798, 1.2910311000305228]


In [12]:
results["exp"] = [list(args), pypy_exp_time]


### Простая хеш-функция для целых чисел

In [13]:
def pypy_hash(x: int) -> int:
    x = ((x >> 16) ^ x) * 0x45d9f3b
    x = ((x >> 16) ^ x) * 0x45d9f3b
    x = (x >> 16) ^ x
    return x


In [14]:
pypy_hash_time = repeat(lambda: pypy_hash(42), repeat=10, number=1000000)
print(pypy_hash_time)

[0.3546615999657661, 0.34810409997589886, 0.34468709997599944, 0.3426058000186458, 0.3420964000397362, 0.3449595000129193, 0.34414410003228113, 0.3453126999665983, 0.35613889998057857, 0.3654800999793224]


In [15]:
xs = range(1, 11)
results["hash"] = [list(xs), pypy_hash_time]


### Числа Фибоначчи

In [17]:
def pypy_fib(n: int) -> int:
    if n < 2:
        return 1
    return pypy_fib(n - 1) + pypy_fib(n - 2)

In [20]:
arg = 30
print(f"fib({arg}) = {pypy_fib(arg)}")
args = range(1, 31)
pypy_fib_time = []
for arg in args:
    pypy_fib_time.append(timeit(lambda: pypy_fib(arg), number=10))
print(pypy_fib_time)

fib(30) = 1346269
[2.00001522898674e-06, 3.500026650726795e-06, 4.599976819008589e-06, 7.199996616691351e-06, 1.1399970389902592e-05, 1.850002445280552e-05, 2.9699993319809437e-05, 4.770001396536827e-05, 7.429998368024826e-05, 0.00012019998393952847, 0.00019869999960064888, 0.000311399984639138, 0.0005107999895699322, 0.0008371000294573605, 0.0013226000010035932, 0.0024093000101856887, 0.003486700006760657, 0.00596199999563396, 0.009394399996381253, 0.015616299991961569, 0.024903199984692037, 0.03809469996485859, 0.0778693999745883, 0.10550129995681345, 0.16616760002216324, 0.2765617999830283, 0.43292499997187406, 0.7023468000115827, 1.1646154000191018, 1.8425163000356406]


In [21]:
results["fib"] = [list(args), pypy_fib_time]

In [25]:
with open("pypy_performance.json", "w", encoding="utf-8") as outfile:
    outfile.write(dumps(results, indent=2))

In [27]:
dumps(results)

'{"sum": [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0.09988090000115335, 0.09813860000576824, 0.09835220000240952, 0.11718719999771565, 0.09999650000827387, 0.09985900000901893, 0.0995898999972269, 0.09989769995445386, 0.09874079999281093, 0.09859180002240464]], "exp": [[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250], [0.12120669998694211, 0.1676558000035584, 0.23789669998222962, 0.2760334999766201, 0.32595530000980943, 0.35333519999403507, 0.4003392999875359, 0.4515677000163123, 0.5425614000414498, 0.555046699999366, 0.6281444999622181, 0.6689620999968611, 0.690308700024616, 0.7371551000396721, 0.7855007000034675, 0.8472960999934003, 0.9347611999837682, 0.9237144000362605, 0.982869400002528, 1.0346267000422813, 1.077914499968756, 1.1484616000088863, 1.1625329999951646, 1.250912800023798, 1.2910311000305228]], "hash": [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0.3546615999657661, 0.34810409997589886, 0.34468709997599944, 0.3426058000